In [ ]:
def pix2radec(ipix_array, nside):
    theta, phi = hp.pix2ang(nside, ipix_array)
    ra = np.rad2deg(phi)
    dec = np.rad2deg(0.5 * np.pi - theta)
    return ra, dec

In [ ]:
def do_dmdt(oid):
    dmdts = []
    dets = detections.loc[oid]
    try: # make sure this is a dataframe and not a series (when only one element)
        dets = dets.to_frame().transpose()
    except:
        True 
    non_dets = non_detections.loc[oid]
    try: # make sure this is a dataframe and not a series (when only one element)
        non_dets = non_dets.to_frame().transpose()
    except:
        True 
    for fid in sorted(dets.fid.unique()):
        if fid == 1:
            fid_str = "g"
        elif fid == 2:
            fid_str = "r"
        else:
            continue
        mask_det = (dets.fid == fid)
        idx_1st_det = dets.loc[mask_det].mjd.astype(float).idxmin()
        mask_non_det = (non_dets.mjd < float(n.loc[oid].firstmjd)) & (non_dets.fid == fid)
        if mask_non_det.sum() >= 1: # filter must be in both dets and non_dets
            dm = np.array(dets.loc[idx_1st_det].magpsf - non_dets.loc[mask_non_det].diffmaglim)
            dm_error = np.array(dets.loc[idx_1st_det].magpsf + dets.loc[idx_1st_det].sigmapsf - non_dets.loc[mask_non_det].diffmaglim)
            dt = np.array(dets.loc[idx_1st_det].mjd - non_dets.loc[mask_non_det].mjd)
            dmdt = dm/dt
            dmdt_error = dm_error/dt
            idx_dmdt = np.argmin(dmdt_error) #dmdt = (dm/dt).min()                                                                                                                                         
            dmdt_min = dmdt[idx_dmdt]
            dmdt_error_min = dmdt_error[idx_dmdt]
            newdf = pd.DataFrame(data = {"oid":[oid],"dmdt_min": [dmdt_min], "dmdt_error": [dmdt_error_min - dmdt_min], \
                                         "dmdt_comp": [dmdt_error_min], 'dt': [dt[idx_dmdt]], 'fid': int(fid), \
                                         'Flag': [dt[idx_dmdt] < 0.5]}, index = [oid])
            dmdts.append(newdf)
            

    if dmdts == []:
        return None
    else:
        
        return pd.concat(dmdts)

In [ ]:
def gethost(name, ra, dec, doplot=False):
    dclient = Delight("./", [name], [ra], [dec])
    dclient.download()
    dclient.get_pix_coords()
    dclient.compute_multiresolution(nlevels=5, domask=False, doobject=True, doplot=False)
    dclient.load_model()
    dclient.preprocess()
    dclient.predict()
    if doplot:
        dclient.plot_host(name)
    display(dclient.df)
    return dclient.df.loc[name].ra_delight, dclient.df.loc[name].dec_delight

In [ ]:
def get_del(oid):
    seloid = oid
    obj = client.query_object(seloid)
    ra_host, dec_host = gethost(seloid, obj["meanra"], obj["meandec"], doplot=False)
    return ra_host,dec_host

In [ ]:
def query(table_delight, table_cand, ned=True, simbad=True):
    start_time = time.time()  # Record the start time

    # Initialize tqdm for the loop
    progress_bar = tqdm(total=len(table_delight))

    # Iterate over rows of both tables with iterrows
    for (index_delight, row_delight), (index_cand, row_cand) in zip(table_delight.iterrows(), table_cand.iterrows()):
        # Convert RA and Dec to degrees
           
        try:
            radelight = row_delight['ra'] * u.degree
            decdelight = row_delight['dec'] * u.degree

            # Create a SkyCoord object
            cobestguess = coordinates.SkyCoord(ra=radelight, dec=decdelight, unit=(u.deg, u.deg), frame='fk5')

            # Initialize dictionary to store cross-matched data
            xmatches = {}

            # Query NED database
            if ned:
                table_ned = customNed.query_region(cobestguess, radius=0.023 * u.deg)
                if table_ned:
                    table_ned["cat_name"] = "NED"
                    xmatches["NED"] = table_ned

            # Query Simbad database
            if simbad:
                table_simbad = customSimbad.query_region(cobestguess, radius=0.023 * u.deg)
                if table_simbad:
                    table_simbad["cat_name"] = "Simbad"
                    xmatches["Simbad"] = table_simbad

            
            a = find_bestguess_host(row_cand, row_delight, xmatches)[0]
            print(tuple(row_delight)[0])
            host[tuple(row_delight)[0]]=a
            # Update the progress bar
            progress_bar.update(1)
            try:
                response = requests.get('http://ned.ipac.caltech.edu', timeout=10)  # Adjust the timeout value as needed
    # Continue processing the response
            except requests.exceptions.Timeout:
                print("The request timed out.")

        except (KeyError,ConnectionError) as e:
            print(f"Error: {e}")
            time.sleep(10.5)
            continue
        
        # Close the progress bar
        progress_bar.close()
        

        end_time = time.time()  # Record the end time
        elapsed_time = end_time - start_time
        print(f"Total time elapsed: {elapsed_time} seconds")
        

In [ ]:
 def process_data(data):

        hostdata = {}
        
        # save clicked information
        if data["cat_name"] != "ZTF":
            hostdata["candidate_host_source"] = data["cat_name"]
        if data["cat_name"] == "NED":
            hostdata["candidate_host_ra"] = data["RA"]
            hostdata["candidate_host_dec"] = data["DEC"]
            hostdata["candidate_host_name"] = data["Object Name"]
            hostdata["candidate_host_redshift_spec"] = False
            if "Redshift" in data.keys():
                if "Redshift Flag" in data.keys():
                    if not str(data["Redshift"]) in ["nan", "-99", "-999.0", "-9999.0", "--"]:
                        hostdata["candidate_host_redshift"] = data["Redshift"]
                        if data["Redshift Flag"] in ["", "SPEC"]:
                            hostdata["candidate_host_redshift_spec"] = True
                        hostdata["candidate_host_redshift_type"] = data["Redshift Flag"]
        elif data["cat_name"] == "Simbad":
            coords = coordinates.SkyCoord("%s %s" % (data["RA"], data["DEC"]), unit=(u.hourangle, u.deg), frame='icrs')
            hostdata["candidate_host_ra"] = coords.ra / u.deg
            hostdata["candidate_host_dec"] = coords.dec / u.deg
            hostdata["candidate_host_name"] = data["MAIN_ID"]
            hostdata["candidate_host_redshift_spec"] = False
            if "Z_VALUE" in data.keys():
                if not data["Z_VALUE"] in ["nan", "-99", "-999.0", "-9999.0"]:
                    hostdata["candidate_host_redshift"] = data["Z_VALUE"]
                    if data["RVZ_ERROR"] != "nan": # based on experience, we only trust Simbad redshifts if they have an associated error
                        hostdata["candidate_host_redshift_spec"] = True
                        if data["RVZ_TYPE"] == "z":
                            hostdata["candidate_host_redshift_error"] = data["RVZ_ERROR"]
                        elif data["RVZ_TYPE"] == "v":
                            cspeed = 299792. # km/s
                            hostdata["candidate_host_redshift_error"] = data["RVZ_ERROR"] / cspeed
                    if "RVZ_QUAL" in data.keys():
                        hostdata["candidate_host_redshift_type"] = data["RVZ_QUAL"]  # this assumes that Simbad only returns spectroscopic redshifts
        elif data["cat_name"] == "SDSSDR16":

            objid = data["objid"]
            
            # long name
            host_name =  "SDSS J%s" % coordinates.SkyCoord(ra=data["ra"]*u.degree, dec=data["dec"]*u.degree, frame='icrs').to_string(style="hmsdms", sep="", pad=True, precision=2)[:-1].replace(" ", "")
            self.info.value =  "Querying SDSSDR16 object %s..." % str(objid)
            if objid not in self.hosts_queried.keys():
                self.hosts_queried[objid] = self.get_SDSSDR16_redshift(objid)
                self.hosts_queried[objid]["host_name"] = host_name

            hostdata["candidate_host_name"] = self.hosts_queried[objid]["host_name"]
            hostdata["candidate_host_ra"] = data["ra"]
            hostdata["candidate_host_dec"] = data["dec"]

            if "specz" in self.hosts_queried[objid].keys():
                if self.hosts_queried[objid]["specz"] in ["nan", "-99", "-999.0", "-9999.0"]:
                    print("Ignoring redshift...")
                else:
                    hostdata["candidate_host_redshift"] = self.hosts_queried[objid]["specz"]
                    hostdata["candidate_host_redshift_spec"] = True
                    hostdata["candidate_host_redshift_error"] = self.hosts_queried[objid]["specz_err"]
                    hostdata["candidate_host_redshift_type"] = "specz"
            elif "photoz" in self.hosts_queried[objid].keys():
                if self.hosts_queried[objid]["photoz"] in ["nan", "-99", "-999.0", "-9999.0"]:
                    print("Ignoring redshift...")
                else:
                    hostdata["candidate_host_redshift"] = self.hosts_queried[objid]["photoz"]
                    hostdata["candidate_host_redshift_error"] = self.hosts_queried[objid]["photoz_err"]
                    hostdata["candidate_host_redshift_spec"] = False
                    hostdata["candidate_host_redshift_type"] = "photoz"
                    
        if not "candidate_host_name" in hostdata.keys():
            hostdata["candidate_host_name"] = "NULL"
        if not "candidate_host_ra" in hostdata.keys():
            hostdata["candidate_host_ra"] = "NULL"
        if not "candidate_host_dec" in hostdata.keys():
            hostdata["candidate_host_dec"] = "NULL"
        if not "candidate_host_redshift" in hostdata.keys():
            hostdata["candidate_host_redshift"] = "NULL"
        if not "candidate_host_redshift_spec" in hostdata.keys():
            hostdata["candidate_host_redshift_spec"] = "NULL"
        if not "candidate_host_redshift_error" in hostdata.keys():
            hostdata["candidate_host_redshift_error"] = "NULL"
        if not "candidate_host_redshift_type" in hostdata.keys():
            hostdata["candidate_host_redshift_type"] = "NULL"
        if not "candidate_host_source" in hostdata.keys():
            hostdata["candidate_host_source"] = "NULL"

        return hostdata

In [ ]:
def find_bestguess_host(table_cand, table_bestguess, xmatches):
        newdf = []
        columns = ["host_name", "host_ra", "host_dec", "host_offset", "host_offset_bestguess", "bestguess_offset", "host_source",
                                        "host_redshift_spec", "host_redshift", "host_redshift_error", "host_redshift_type"]
        for key in xmatches.keys():
            for i in xmatches[key]:
                hostdata = process_data(i)
                if hostdata["candidate_host_ra"] != "NULL" and hostdata["candidate_host_dec"] != "NULL":
                    hostdata["candidate_host_offset"] = coordinates.SkyCoord(hostdata["candidate_host_ra"] * u.deg, hostdata["candidate_host_dec"] * u.deg, frame='icrs').separation(
                        coordinates.SkyCoord(float(table_cand["RA"]) * u.deg, float(table_cand["DEC"]) * u.deg, frame='icrs')).arcsecond
                    hostdata["candidate_host_offset_bestguess"] = coordinates.SkyCoord(hostdata["candidate_host_ra"] * u.deg, hostdata["candidate_host_dec"] * u.deg, frame='icrs').separation(
                        coordinates.SkyCoord(float(table_bestguess['ra']) * u.deg, float(table_bestguess["dec"]) * u.deg, frame='icrs')).arcsecond
                    hostdata["candidate_bestguess_offset"] = coordinates.SkyCoord(float(table_cand["RA"]) * u.deg, float(table_cand["DEC"]) * u.deg, frame='icrs').separation(
                        coordinates.SkyCoord(float(table_bestguess['ra']) * u.deg, float(table_bestguess["dec"]) * u.deg, frame='icrs')).arcsecond
                else:
                    hostdata["candidate_host_offset"] = "NULL"
                    
                newdf.append(pd.DataFrame([[hostdata["candidate_host_name"],
                               hostdata["candidate_host_ra"],
                               hostdata["candidate_host_dec"],
                               hostdata["candidate_host_offset"],
                               hostdata["candidate_host_offset_bestguess"],
                               hostdata["candidate_bestguess_offset"],
                               hostdata["candidate_host_source"],
                               hostdata["candidate_host_redshift_spec"],
                               hostdata["candidate_host_redshift"],
                               hostdata["candidate_host_redshift_error"],
                               hostdata["candidate_host_redshift_type"]]],
                                          columns = columns,
                                          index = [str(table_cand["ZTF_oid"][0])]))

        if newdf != []:
            newdf = pd.concat(newdf).sort_values("host_offset_bestguess")
        else:
            newdf = pd.DataFrame([], columns=columns, index = [str(table_cand["ZTF_oid"][0])])
        newdf.replace(["NULL", "nan", -99, -999, -9999], np.nan, inplace=True)
        newdf.index.name = "oid"

        # otherwise, the closest
        min_bestguess_offset = newdf.iloc[0].host_offset_bestguess
        min_host_offset = newdf.iloc[0].host_offset
        bestguess_offset = newdf.iloc[0].bestguess_offset
        nmin_offset = 4
        # spectroscopic redshifts with error
        mask_specz_err = newdf.host_redshift_spec & (newdf.host_redshift_error > 0)
        if mask_specz_err.sum() > 0:
            min_bestguess_offset_specz_err = newdf.loc[mask_specz_err].iloc[0].host_offset_bestguess
            min_offset_specz_err = newdf.loc[mask_specz_err].iloc[0].host_offset
            # the distance from the predicted position and from the candidate cannot be nmin_offset times the minimum distance from the predicted position and the candidate, respectively
            if (min_bestguess_offset_specz_err < nmin_offset * max(1, min_bestguess_offset)) and (min_offset_specz_err < nmin_offset * max(1, bestguess_offset)):
                return newdf.loc[mask_specz_err].iloc[0]
        # spectroscopic redshifts
        mask_specz = newdf.host_redshift_spec
        if mask_specz.sum() > 0:
            min_bestguess_offset_specz = newdf.loc[mask_specz].iloc[0].host_offset_bestguess
            min_offset_specz = newdf.loc[mask_specz].iloc[0].host_offset
            if (min_bestguess_offset_specz < nmin_offset * max(1, min_bestguess_offset)) and (min_offset_specz < nmin_offset * max(1, bestguess_offset)):
                return newdf.loc[mask_specz].iloc[0]
        # photometric redshifts
        mask_photoz = newdf.host_redshift_type == "photoz"
        if mask_photoz.sum() > 0:
            min_bestguess_offset_photoz = newdf.loc[mask_photoz].iloc[0].host_offset_bestguess
            min_offset_photoz = newdf.loc[mask_photoz].iloc[0].host_offset
            if (min_bestguess_offset_photoz < nmin_offset * max(1, min_bestguess_offset)) and (min_offset_photoz < nmin_offset * max(1, bestguess_offset)):
                return newdf.loc[mask_photoz].iloc[0]
        # if no redshift with previus conditions, check if nearest source if close enough to candidate
        if min_host_offset < nmin_offset * max(1, bestguess_offset):
            return newdf.iloc[0]
        else:
            print("WARNING: closest crossmatch is too far from the predicted position. No catalog crossmatches found.")
            display(newdf.fillna("NULL"))
            aux = pd.DataFrame(newdf.iloc[0])
            aux.loc[:] = np.nan
            return aux

In [ ]:
import math
def ell_dist(A,B,pa,ra,dec,x,y):
    cxx=(math.cos(pa)**2/A**2)+(math.sin(pa)**2/B**2)
    cyy=(math.sin(pa)**2/A**2)+(math.cos(pa)**2/B**2)
    cxy=2*math.cos(pa)*math.sin(pa)*((1/A**2)-(1/B**2))
    cx=ra
    cy=dec 
    return cxx * (x - cx)**2 + cxy * (y - cy) * (x - cx) + cyy * (y - cy)**2